# Quadruple Line Junctions

Objective: Calculate the sum of shear to obtain rigid body displacement.

### Import packages

In [1]:
## numpy: package for cumputing numerical arrays
from numpy.linalg import inv #inverse of a matrix
import numpy as np
np.set_printoptions(precision=4) #precision of arrays
#i have set a custom threshold of 9 sigfigs after decimal

## matplotlib: package for plotting
from matplotlib import pyplot as plt
%matplotlib inline

## import functions from twinning
from twinning import *

### 10M Ni-Mn-Ga

1. Lattice parameters
2. Metric Tensor
3. Irrational element of NC twins

In [2]:
## lattice parameters
a,b,c,gamma = [0.5972, 0.5944, 0.5584, np.deg2rad(90.37)] #NMG lattice parameter: Seiner et al. (2019)
#a,b,c,gamma = [1., 0.97, 0.94, np.deg2rad(94)] #Exaggerated lattice

## metric tensor
G = np.array([[a**2,a*b*np.cos(gamma),0],[a*b*np.cos(gamma),b**2,0],[0,0,c**2]])

## irrational element of NC twins
q3 = (2*a*b*np.cos(gamma) - np.sqrt(a**4 + b**4 + 2*a**2*b**2*np.cos(2*gamma)))/(a**2 - b**2)
q4 = (2*a*b*np.cos(gamma) + np.sqrt(a**4 + b**4 + 2*a**2*b**2*np.cos(2*gamma)))/(a**2 - b**2)
#q3 * q4 ≃ -1

### Monoclinic Composite forming Quadrupole Line

| Variant Transformation | $\boldsymbol{K}_1$ | $\boldsymbol{\eta}_1$ | Orientation Relationship |
| ---------------------: | :----------------: | :-------------------: | ------------------------: |
| $V_1 \to V_3$          |  $(1 1 0)_m$       | $[\bar{1} 1 0]_m$   | $\boldsymbol{2}_{[\bar{1}10]_m}$ |
| $V_3 \to V_2$          |  $(1 q_4 0)_m$       | $[\bar{q_4} 1 0]_m$   | $\sim \boldsymbol{4}^+_{[001]_m}$ |
| $V_2 \to V_4$          |  $(\bar{1} \bar{1} 0)_m$       | $[1 \bar{1} 0]_m$   | $\boldsymbol{2}_{[1 \bar{1} 0]_m}$ |
| $V_4 \to V_1$          |  $(1 q_4 0)_m$       | $[\bar{q_4} 1 0]_m$   | $\sim \boldsymbol{4}^+_{[001]_m}$ |



#### $^R\boldsymbol{P}^{V_1}$

In [3]:
## Transformation: R ↔ V₁
rPv1 = np.array([[1,b/a*np.cos(gamma),0],[0,-b/a*np.sin(gamma),0],[0,0,-c/a]])

#### $^R\boldsymbol{P}^{V_3}$

In [4]:
## Transformation: R ↔ V₃
#OR (1 1 0) compound twin
rL_v1tov3 = rotationmatrix([1,1,0] @ inv(rPv1), np.pi)
rPv3 = rL_v1tov3 @ rPv1
del rL_v1tov3 #del OR

#### $^R\boldsymbol{P}^{V_2}$

In [5]:
## Transformation: R ↔ V₂
#OR (1 q4 0) NC twin
rL_v3tov2 = OR_NCtwins([1,q4,0],[-q4,1,0],rPv3,a,normal_pos=True)
rPv2 = rL_v3tov2 @ rPv3
del rL_v3tov2 #del OR

#### $^R\boldsymbol{P}^{V_4}$

In [6]:
## Transformation: R ↔ V₄
#OR (-1 -1 0) compound twin
rL_v2tov4 = rotationmatrix([-1,-1,0] @ inv(rPv2), np.pi)
rPv4 = rL_v2tov4 @ rPv2
del rL_v2tov4 # del OR

### Sum of Shears (Rigid Body Displacement)

#### shear magnitude

In [7]:
##define output variable
shear = []

## shear magnitude from C

## (1 1 0) TB Correspondence
C = applythresh(rotationmatrix([1,1,0],np.pi))
#shear of compound twin
shear.append(np.sqrt(np.trace(C.T @ G @ C @ inv(G))-3))
del C

## (1 1 0) TB Correspondence
C = applythresh(rotationmatrix([0,0,1],np.pi/2))
#shear of compound twin
shear.append(np.sqrt(np.trace(C.T @ G @ C @ inv(G))-3))
del C

#print
print('shear magnitdue: (110) and (1 q3 0) TB = ', np.asarray(shear))

shear magnitdue: (110) and (1 q3 0) TB =  [0.0094 0.016 ]


#### $\boldsymbol{s}\ \colon\ V_1 \to V_3$

In [8]:
##define output variable
sheardirection = []

## V₁ → V₃
eta1 = np.array([-1,1,0])
sd = rPv1 @ eta1
del eta1

## unit shear direction * magnitude
sd = shear[0] * (sd / np.sqrt(sd.T @ sd))

#print & remove
print('shear1 = ', sd)
sheardirection.append(sd);
del sd

shear1 =  [-0.0067 -0.0066  0.    ]


#### $\boldsymbol{s}\ \colon\ V_3 \to V_2$

In [9]:
## V₃ → V₂
eta1 = np.array([-q4,1,0])
sd = rPv3 @ eta1
del eta1

## unit shear direction * magnitude
sd = shear[1] * (sd / np.sqrt(sd.T @ sd))

#print & remove
print('shear3 = ', sd)
sheardirection.append(sd);
del sd

shear3 =  [0.0152 0.0049 0.    ]


#### $\boldsymbol{s}\ \colon\ V_2 \to V_4$

In [10]:
## V₂ → V₄
eta1 = np.array([1,-1,0])
sd = rPv2 @ eta1
del eta1
## unit shear direction * magnitude
sd = shear[0] * (sd / np.sqrt(sd.T @ sd))

#print & remove
print('shear2 = ', sd)
sheardirection.append(sd);
del sd

shear2 =  [ 0.0066 -0.0067  0.    ]


#### $\boldsymbol{s}\ \colon\ V_4 \to V_1$

In [11]:
## V₃ → V₂
eta1 = np.array([-q4,1,0])
sd = rPv4 @ eta1
del eta1

## unit shear direction * magnitude
sd = shear[1] * (sd / np.sqrt(sd.T @ sd))

#print & remove
print('shear4 = ', sd)
sheardirection.append(sd);
del sd

shear4 =  [ 0.0151 -0.0052  0.    ]


### Sum of shears

In [12]:
sumofshear = sum(sheardirection)
magn = np.sqrt(sumofshear.T @ sumofshear)
dire = sumofshear / magn
del sumofshear
print('shear magnitude = %.4f' % magn)
print('shear direction = ', dire)

shear magnitude = 0.0331
shear direction =  [ 0.9114 -0.4115  0.    ]
